<a href="https://colab.research.google.com/github/Tian1398/INST377-SP2021/blob/main/G4_application_download_companySECdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I used SEC API to create a small application that automately download the relevent data in csv file as the user enters the company's stock symbol (ticker name).

In [1]:
import requests
import pandas as pd
from google.colab import files

# Fetch CIK

In [29]:
# set headers
headers = {'User-Agent': "tiancz23@uw.edu"}

In [37]:
# retrieve company CIKs so you can make the data to each company from other documents
tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
# check if the scripting went thru
#print(tickers_cik.json().keys())

# organize data, make CIKs to 10 digits
company_data = pd.DataFrame.from_dict(tickers_cik.json(), orient="index")#.reset_index()
company_data["cik_str"] = company_data["cik_str"].astype(str).str.zfill(10)
company_data.head()

,cik_str,ticker,title
0,0000789019,MSFT,MICROSOFT CORP
1,0000320193,AAPL,Apple Inc.
2,0001045810,NVDA,NVIDIA CORP
3,0001652044,GOOGL,Alphabet Inc.
4,0001018724,AMZN,AMAZON COM INC


# Write function to get company's asset data by name

In [72]:
def download_company_data(ticker_name):
  # ask for input
  ticker_name = input("Please enter a comapny's stock symbol (tick): ")
  ticker_name = ticker_name.upper()
  # locate cik bt company ticker
  cik_str = company_data.loc[company_data["ticker"] == ticker_name, "cik_str"].values
  for each in cik_str:
    url = f"https://data.sec.gov/api/xbrl/companyconcept/CIK{each}/us-gaap/Assets.json"
    response = requests.get(url, headers=headers)
    # if asset data is successfully parsed:
    if response.status_code == 200:
      assets_timeserie = pd.json_normalize(response.json()["units"]["USD"])
      assets_timeserie["filed"] = pd.to_datetime(assets_timeserie["filed"])
      assets_timeserie = assets_timeserie.sort_values("end")
      # download data in csv format
      assets_timeserie.to_csv("CompanyAssetData.csv", index = False)
      return files.download("CompanyAssetData.csv")
    # print error if data is not found
    else:
      print(f"Failed to fetch data for company: {ticker_name}.")
      return None

In [73]:
# test function
download_company_data()

Please enter a comapny's stock symbol (tick): AMZN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>